In [38]:
#Para hacer scraping de la página del Boletín oficial (búsqueda avanzada)
#Selenium es un conjunto de herramientas de software de código abierto utilizado para la 
#automatización de pruebas funcionales en aplicaciones web. Selenium proporciona una 
#interfaz de programación de aplicaciones (API) para controlar un navegador web y simular 
#la interacción humana con una página web.

#Importar librerías
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

#Lista vacía
dfs = []

# configurar el webdriver
service = Service('./chromedriver')
driver = webdriver.Chrome(service=service)

# abrir la página web
driver.get("https://www.boletinoficial.gob.ar/busquedaAvanzada/primera")

#Definir temas a buscar (en el campo palabra clave). La idea es buscar por tema, sin definir tipo de norma
temas= ['precios'
        , 'multas'
        , 'agio'
        , 'especulación'
        , 'abastecimiento'
#        , 'inflación'
       ]

#Definir años de búsqueda
año_inicio = 1900
año_fin = 1976
años = list(range(año_inicio, año_fin))
años = list(map(str, años))

# buscar elementos 
#itera sobre los temas
for j in temas:
    #Para cada tema, si lo encuentra, intenta iterar sobre los años
    for i in años:
    
        # Cambo de búsqueda palabra clave
        search_box1 = driver.find_element(By.ID, "palabraClave")
        search_box1.send_keys(j)

        # Cambo de búsqueda palabra año
        search_box2 = driver.find_element(By.ID, "anioNormaIP")
        search_box2.send_keys(i)
     
        try:
            # hacer clic en el botón de búsqueda
            search_button = WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.ID, "btnBusquedaAvanzada")))
            search_button.click()
            
            # espera a que se carguen nuevos resultados (1 segundo)
            time.sleep(1)
            
            # limpiar ambos campos de entrada
            search_box1.clear()
            search_box2.clear()
           
            
            # hacer scroll hasta el final de la página
            while True:
                # obtener número actual de resultados
                num_resultados = len(driver.find_elements(By.XPATH, "//div[@class='list-group-item list-group-item-action flex-column align-items-start']"))
    
                # hacer scroll hasta el final de la página
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
                # esperar a que se carguen nuevos resultados (2 segundos)
                time.sleep(2)
    
                # obtener nuevo número de resultados
                num_nuevos_resultados = len(driver.find_elements(By.XPATH, "//div[@class='list-group-item list-group-item-action flex-column align-items-start']"))
    
                # salir del bucle si no hay más resultados nuevos
                if num_nuevos_resultados == num_resultados:
                    break

            # extraer resultados visibles
            resultados = driver.find_elements(By.XPATH, "//div[contains(@class, 'col-md-12') and @class!='row']")

            #Busca el elemento "8" entre los resultados, que es el que contiene toda la normativa
            normas = resultados[8].text
            #Separa la gran string de normas en una lista
            lista_normas = normas.split('\n')

            #Se eliminan de la lista los elementos que indican los tipos de normas que siguen
            for item in ['LEYES', 'DECRETOS', 'RESOLUCIONES', 'DISPOSICIONES', 
                         'RESOLUCIONES GENERALES', 'RESOLUCIONES CONJUNTAS', 'AVISOS OFICIALES']:
                if item in lista_normas:
                    lista_normas.remove(item)
            
            #Se crean 4 columnas con los 4 elementos de cada norma
            original = lista_normas
            n = 4
            sublistas = []

            for i in range(0, len(original), n):
                sublistas.append(original[i:i+n])

            # Crear la lista de listas
            lista_de_listas = sublistas

            # Crear el DataFrame
            df = pd.DataFrame(data=lista_de_listas, columns=['Poder', 'Norma', 'Fecha', 'Contenido'])
            df['tema'] = j

            dfs.append(df)
        
        except:
            # si no se encuentra la palabra clave, pasar a la siguiente iteración sin generar un mensaje de error
            pass

In [ ]:
#crea un dataframe con las normas ya divididas en 4 columnas
dfs_concatenado = pd.concat(dfs)
dfs_concatenado = dfs_concatenado.reset_index()
dfs_concatenado = dfs_concatenado.drop('index', axis=1)

In [41]:
#Procesamiento de lenguaje natural con spacy
#SpaCy es una biblioteca de procesamiento del lenguaje natural (NLP) 
#de código abierto para Python. SpaCy se utiliza para realizar tareas 
#de procesamiento de texto, como tokenización, análisis sintáctico, 
#etiquetado POS (partes del habla), reconocimiento de entidades nombradas, 
#desambiguación de sentidos, entre otras.

#SpaCy también cuenta con modelos pre-entrenados para varios idiomas, 
#incluidos inglés, español, alemán, francés, portugués, italiano y holandés, 
#lo que permite a los usuarios realizar tareas de NLP sin necesidad de entrenar modelos desde cero.

In [40]:
from math import isnan
from unidecode import unidecode


In [42]:
# Cargar modelo de lenguaje en español
nlp = spacy.load('es_core_news_sm')

In [43]:
import spacy
# Definir función para analizar texto
def analizar_texto(texto):
    doc = nlp(texto)
    #usar el modelo para extraer diversas entidades del resumen del contenido de cada norma 
    #Obtener sustantivos y nombres propios, verbos y adjetivos
    sustantivos_y_nombres_propios = [token.lemma_ for token in doc if token.pos_ in ['NOUN', 'PROPN']]
    verbos = [token.lemma_ for token in doc if token.pos_ == 'VERB']
    adjetivos = [token.text for token in doc if token.pos_ == 'ADJ']
    # Encontrar caracteres numéricos 
    #(que normalmente representan otras normas a las que hace referencia la que estamos analizando)
    numeros = [token.text for token in doc if token.is_digit]
    return sustantivos_y_nombres_propios, verbos, adjetivos, numeros

#crear 4 columnas nuevas con las distintas entidades aplicando la función a la columna 'contenido' 
dfs_concatenado['Contenido'].fillna('', inplace=True)
dfs_concatenado[['sustantivos_y_nombres_propios', 'verbos', 'adjetivos', 'numeros']] = pd.DataFrame(dfs_concatenado['Contenido'].apply(analizar_texto).tolist())

In [ ]:
#mejorar algunas columnas
dfs_concatenado[['Norma', 'Año']] = dfs_concatenado['Norma'].str.split('/', n=1, expand=True)
dfs_concatenado[['Norma', 'Número']] = dfs_concatenado['Norma'].str.split(' ', n=1, expand=True)
dfs_concatenado['Fecha'] = dfs_concatenado['Fecha'].str.replace('Fecha de Publicacion: ', '')

In [61]:
dfs_concatenado["Año"] = dfs_concatenado["Año"].str.replace("N/", "")
dfs_concatenado["Año"] = dfs_concatenado["Año"].str.replace("n/", "")

dfs_concatenado['Año'] = pd.to_numeric(dfs_concatenado['Año'] , errors='coerce').astype('Int64')

In [74]:
#guardar dataset con información categorizada de normas de diversos temas y 
#con la amplitud de fechas que queramos

dfs_concatenado.to_excel('normas.xlsx', index=False)



## FIN